# 텍스트 표현(Representing text)

신경망으로 자연어처리(NLP) 작업을 해결하려면 텍스트를 텐서로 나타낼 수 있는 방법이 필요하다. 컴퓨터는 이미 ASCII 또는 UTF-8와 같은 인코딩을 사용하여 화면의 폰트에 매핑되는 숫자로 텍스트 문자를 나타냅니다.
If we want to solve Natural Language Processing (NLP) tasks with neural networks, we need some way to represent text as tensors. Computers already represent textual characters as numbers that map to fonts on your screen using encodings such as ASCII or UTF-8.

![ASCII 및 이진 표현에 문자를 매핑하는 다이어그램 표시 이미지](./images/ascii-character-map.png)

우리는 각 글자가 무엇을 나타내는지(**represents**), 그리고 어떻게 모든 글자가 함께 모여 문장의 단어를 형성하는지 이해한다. 그러나, 컴퓨터 자체는 그러한 이해를 가지고 있지 않으며, 신경망은 훈련 중에 그 의미를 배워야 한다.

따라서 텍스트를 나타낼 때 서로 다른 접근 방식을 사용할 수 있습니다:
* **문자 수준 표현(Character-level representation)** : 각 문자를 숫자로 처리하여 텍스트를 나타낼 때. 텍스트 말뭉치(corpus)에 $C$개의 다른 문자가 있는 경우, *Hello* 라는 단어는 $5\times C$ 텐서로 표현된다. 각 문자는 원-핫 인코딩의 텐서 컬럼에 해당된다.
* **단어 수준 표현(Word-level representation)**, : 텍스트에 있는 모든 단어의 **사전(vocabulary)** 을 만든 다음 원-핫 인코딩을 사용하여 단어를 나타냅니다. 이 접근 방식은 각 문자(Character) 자체에는 큰 의미가 없기 때문에 다소 더 나은 편이며, 따라서 상위 수준의 의미 개념인 단어(Word)를 사용함으로써 신경망에 대한 작업을 단순화한다. 그러나 큰 사전(vocabulary) 크기를 고려할 때, 우리는 고차원 희소 텐서를 다룰 필요가 있다.

이 모듈에서 사용할 필수 Python 패키지를 설치하는 것부터 시작하겠습니다.

##### !pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

# 텍스트 분류 작업(Text classification task)

이 모듈에서는 뉴스 헤드라인 토픽을 4가지 클래스(World, Sports, Business and Sci/Tech) 중 하나로 분류하는 **AG_NEWS** 데이터셋을 기반으로 하는 간단한 텍스트 분류 작업부터 시작한다. . 이 데이터 세트는 [`torchtext`](https://github.com/pytorch/text) 모듈에 내장되어 있으므로 쉽게 액세스할 수 있다.

In [39]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

여기서 `train_dataset`과 `test_dataset`은 각각 라벨(클래스 수)과 텍스트의 쌍을 반환하는 반복자(iterators)를 포함한다. 예를 들면,

In [40]:
next(train_dataset)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")


이제 데이터 세트에서 처음 10개의 새로운 헤드라인을 출력해 보겠습니다.

In [41]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace ba

데이터 세트는 반복되므로 데이터를 여러 번 사용하려면 리스트(list)로 변환해야 합니다:

In [42]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

이제 텍스트를 텐서로 나타낼 수 있는 **숫자(numbers)** 로 변환해야 합니다. 단어 수준(Word-level)을 표현하려면 두 가지 작업을 수행해야 합니다.

* 텍스트를 **토큰(tokens)** 으로 분리하려면 **tokenizer** 를 사용해야 한다.
* 토큰의 **사전(vocabulary)** 으로 구축한다.

In [43]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [44]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.Vocab(counter, min_freq=1)

사전을 사용하여 토큰화된 문자열을 다음과 같은 숫자 집합으로 쉽게 인코딩할 수 있다:

In [45]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95812


[283, 2321, 5, 337, 19, 1301, 2357]

## Bag of Words(BoW) 텍스트 표현

단어는 뜻을 나타내기 때문에, 우리는 때때로 문장의 순서와 상관없이 각각의 단어들을 보는 것만으로도 텍스트의 의미를 알아낼 수 있다. 예를 들어, 뉴스를 분류할 때, *날씨*, *눈* 과 같은 단어는 *날씨예보*를 나타낼 가능성이 높은 반면 *주식*, *달러* 와 같은 단어는 *금융뉴스*에 해당됩니다.


**Bag of Words** (BoW) 벡터 표현은 가장 일반적으로 사용되는 벡터 표현이다. 각 단어는 벡터 인덱스에 연결되며, 벡터 내 원소는 주어진 문서에서 단어의 발생 횟수를 포함합니다. 

![BoW 벡터 표현이 메모리에 어떻게 표현되는지 보여주는 이미지.](./images/bag-of-words-example.png) 

> **참고**: 또한 BoW를 텍스트에서 개별 단어에 대한 모든 원-핫 인코딩 벡터의 합으로 생각할 수 있습니다

다음은 Scikit Learn python 라이브러리를 사용하여 BoW 단어 표현을 생성하는 방법의 예입니다.

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()



array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

AG_NEWS 데이터셋의 벡터 표현에서 BoW 벡터를 계산하려면 다음 함수를 사용할 수 있다:

In [47]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([0., 0., 2.,  ..., 0., 0., 0.])


> **참고:** 여기서는 전역 변수 `vocab_size` 를 사용하여 사전의 기본 크기를 지정한다. 종종 사전의 크기는 꽤 크기 때문에 가장 빈번하게 나타나는 단어로 제한할 수 있다. `vocab_size` 값을 낮추고 아래의 코드를 실행하여 모델 정확도에 어떤 영향을 미치는지 확인하십시오. 성능이 더 높은 대신 정확도가 다소 떨어질 것으로 예상해야 합니다.

##  BoW 분류기 학습하기

이제 텍스트에 대하여 BoW 표현을 구축하는 방법을 배웠으니, 여기에 분류기(classifier)를 추가해 보겠습니다. 먼저, 우리는 모든 위치 벡터(positional vector) 표현이 BoW 표현으로 변환되도록 훈련 데이터셋을 변환해야 한다. 이는 `bowify` 함수를 `collate_fn` 매개 변수로 표준 토치 `DataLoader`에 전달하면 달성할 수 있다.

In [48]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

이제 하나의 선형 레이어를 포함하는 간단한 분류기 신경망을 정의해 봅시다. 입력 벡터의 크기는 `vocab_size`와 같으며 출력 크기는 클래스 수인 4에 해당합니다. 분류 작업을 해결하기 때문에 최종 활성화 함수는 `LogSoftmax()`이다. 

In [49]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

이제 표준 PyTorch 훈련 루프를 정의하겠습니다. 우리의 데이터셋은 상당히 크기 때문에, 학습 목적을 위해 우리는 단 한 번의 epoch 동안만 훈련할 것이고, 때로는 1 epoch 미만의 기간 동안 훈련할 것이다(`epoch_size` 파라미터를 지정하면 훈련을 제한할 수 있다). 또한 훈련 중 누적된 훈련 정확도를 보고한다. 보고 빈도는 `report_freq` 파라미터를 사용하여 지정된다.

In [50]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [51]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.80875
6400: acc=0.845625
9600: acc=0.8625
12800: acc=0.86875


(0.02492899782876216, 0.8724013859275054)

## BiGrams, TriGrams and N-Grams

BoW 접근방식의 한 가지 한계점은 어떤 단어들은 합성어라는 것이다. 예를 들어, 'hot dog'라는 단어는 다른 문맥 안에서 'hot'과 'dog'라는 단어와는 완전히 다른 의미를 갖는다. 만약 우리가 항상 같은 벡터로 단어 'hot'과 'dog'를 표현한다면, 그것은 우리의 모델을 혼란스럽게 할 수 있다.

이를 해결하기 위하여 **N-gram representations** 은 문서 분류 방법에 자주 사용되며, 여기서 각 단어, bi-word 또는 tri-word의 빈도는 분류기를 훈련시키는 데 유용한 기능이다. 예를 들어, bigram 표현에서 우리는 모든 단어 쌍을 원래 단어 외에 사전에 추가할 것이다.

다음은 Scikit Learn을 사용하여 bigram BoW 표현을 생성하는 방법의 예입니다.

In [52]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

N-gram 접근법의 주요 단점은 사전(vocabulary) 크기가 극도로 빠르게 증가하기 시작한다는 것이다. 실제로, 우리는 다음 유닛에서 논의할 임베딩과 같은 일부 차원 축소 기법과 N-gram 표현을 결합해야 한다.

**AG News** 데이터셋에서 N-gram 표현을 사용하려면 특별한 N-gram 사전을 구축해야 한다:


In [56]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.Vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308844


그런 다음 위와 동일한 코드를 사용하여 분류기를 훈련시킬 수 있지만, 메모리 효율성이 매우 떨어진다. 다음 단원에서는 임베딩을 사용하여 bigram 분류기를 훈련할 것이다.

> **참조:** 텍스트에서 발생하는 ngrams은 지정된 횟수보다 많이 남겨둘 수 있습니다. 이렇게 하면 간헐적인 bigrams이 생략되고 차원이 크게 감소합니다. 이렇게 하려면 `min_freq` 매개 변수를 더 높은 값으로 설정하고 사전 변화의 길이를 관찰한다.


## Term Frequency Inverse Document Frequency TF-IDF

BoW 표현에서 단어 발생은 단어 자체에 관계없이 균등하게 가중된다. 그러나 a, in 과 같은 빈번한 단어들은 전문 용어들보다 분류에 훨씬 덜 중요하다는 것은 분명하다. 사실 대부분의 NLP 작업에서 특정 단어는 이 외의 단어보다 더 관련이 있다.

**TF-IDF** 는 **term frequency–inverse document frequency** 를 의미한다. BoW의 변형으로, 문서 내에 있는 단어의 모양을 나타내는 이진수 0/1 대신 말뭉치(corpus)에서 단어 발생 빈도와 관련된 부동 소수점 값이 사용된다.

보다 공식적으로, 문서 $j$ 에서 단어 $i$ 의 가중치 $w_{ij}$ 는 다음과 같이 정의된다:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
여기서
* $tf_{ij}$ 는 문서에서 특정 단어의 발생 횟수, BoW 값과 같다.
* $N$ 는 컬렉션에서의 문서의 수
* $df_i$ 는 전체 컬렉션에서 특정 단어가 포함된 문서의 수

TF-IDF 값 $w_{ij}$ 은 문서에 단어가 나타나는 횟수에 비례하여 증가하며, 단어를 포함하는 말뭉치의 문서 수에 의해 상쇄되므로 일부 단어가 다른 단어보다 더 자주 나타나도록 조정할 수 있다. 예를 들어, 만약 그 단어가 컬렉션의 *모든* 문서 $df_i=N$ 및 $w_{ij}=0$에 나타난다면, 그 용어들은 완전히 무시될 것이다.

Scikit Learn을 사용하여 텍스트의 TF-IDF 벡터화를 쉽게 만들 수 있습니다:

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

그러나 TF-IDF 표현은 서로 다른 단어에 주파수 가중치를 제공하지만 단어의 의미나 순서를 나타낼 수 없다. 1935년 유명한 언어학자 J. R. Firth가 말했듯이, "단어의 완전한 의미는 항상 문맥적이어서 문맥을 제외한 의미에 대한 연구는 심각하게 받아들여질 수 없다" 우리는 이후 섹션에서 언어 모델링을 사용하여 텍스트에서 상황 정보(contextual information)를 캡처하는 방법에 대해 배울 것이다.